In [1]:
!pip3 install -r requirements.txt

In [2]:
import os
from utilities.data_utils.Dataset import FacialDataset, get_transform
from utilities.utils import collate_fn
from utilities.train_eval.engine import train_one_epoch, evaluate, get_model_result
import glob

import nvidia_smi # for python 3, you need nvidia-ml-py3 library

import torch
torch.cuda.empty_cache()
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [3]:
PATH = 'Weight.pth'

output_image_folder = 'output'

proj = 'sign' # 'sign','fabric' (put ur project name here)

if proj == 'sign':
    classes = ['warning', 'prohibitory', 'mandatory']
elif proj == 'fabric':
    classes = ['Hole', 'Line', 'Stain']
else:
    classes = ['bg']

num_classes = len(classes)+1  # n class + background

batch_size = 3

num_epochs = 30

In [4]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

Total memory: 8505131008
Free memory: 8067940352
Used memory: 437190656


In [5]:
dataset_train = FacialDataset('data/train', get_transform(horizontal_flip=True),classes=classes)
dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_train = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0,
        collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [7]:
anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                        aspect_ratios=([1.0, 1.0, 1.0, 1.0], 
                                                     [0.8, 1.0, 1.0, 1.0], 
                                                     [1.0, 0.8, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0]))

In [8]:
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [9]:
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Training

In [10]:
for epoch in range(num_epochs):
    res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    print(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

print("Training complete!")

gpu: 33%, gpu-mem: 3%
Epoch: [0]  [ 0/11]  eta: 0:00:12  lr: 0.000505  loss: 2.2714 (2.2714)  loss_classifier: 0.0256 (0.0256)  loss_box_reg: 0.0066 (0.0066)  loss_objectness: 0.5678 (0.5678)  loss_rpn_box_reg: 1.6715 (1.6715)  time: 1.1364  data: 0.1602  max mem: 3745
Epoch: [0]  [ 1/11]  eta: 0:00:11  lr: 0.001004  loss: 1.6878 (1.9796)  loss_classifier: 0.0256 (0.0629)  loss_box_reg: 0.0066 (0.0204)  loss_objectness: 0.5547 (0.5612)  loss_rpn_box_reg: 0.9987 (1.3351)  time: 1.1069  data: 0.2103  max mem: 4015
Epoch: [0]  [ 2/11]  eta: 0:00:10  lr: 0.001503  loss: 2.2714 (2.1540)  loss_classifier: 0.0256 (0.0495)  loss_box_reg: 0.0066 (0.0143)  loss_objectness: 0.5678 (0.5885)  loss_rpn_box_reg: 1.6715 (1.5017)  time: 1.1658  data: 0.3053  max mem: 4015
Epoch: [0]  [ 3/11]  eta: 0:00:09  lr: 0.002003  loss: 2.2714 (2.2578)  loss_classifier: 0.0228 (0.0401)  loss_box_reg: 0.0022 (0.0108)  loss_objectness: 0.5678 (0.5841)  loss_rpn_box_reg: 1.6715 (1.6227)  time: 1.1942  data: 0.3394  

In [11]:
# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)

## Evaluate

In [12]:
evaluate(model, data_loader_test, device=device)

n_threads =  1
creating index...
index created!
Test:  [0/6]  eta: 0:00:00  model_time: 0.0972 (0.0972)  evaluator_time: 0.0005 (0.0005)  time: 0.1308  data: 0.0316  max mem: 4015
Test:  [5/6]  eta: 0:00:00  model_time: 0.1005 (0.1044)  evaluator_time: 0.0008 (0.0008)  time: 0.1346  data: 0.0274  max mem: 4015
Test: Total time: 0:00:00 (0.1347 s / it)
Averaged stats: model_time: 0.1005 (0.1044)  evaluator_time: 0.0008 (0.0008)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.034
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.086
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.045
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     

In [13]:
torch.cuda.synchronize()
# create directory for saving the model
print("Saving model...")
torch.save(model.state_dict(), PATH)
print("Model saving complete!")
nvidia_smi.nvmlShutdown()

Saving model...
Model saving complete!


## Inference 
Repeating all the process to ensure nothing is taken from training stage
Check output folder for results

In [14]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

dataset_test = FacialDataset('data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                   aspect_ratios=([1.0, 1.0, 1.0, 1.0],
                                                  [0.8, 1.0, 1.0, 1.0],
                                                  [1.0, 0.8, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0]))
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load(PATH))
model.to(device)

# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)
# write testing result to output folder
for img_idx, batch_sampler in enumerate(data_loader_test):
    img_test = batch_sampler[0][0]
    target_test = batch_sampler[1][0]
    i = target_test["image_id"].item()
    get_model_result(img_test, model, target_test, i, device, location=output_image_folder, threshold=0.15,classes=classes)

print("Testing complete!")

torch.cuda.synchronize()
nvidia_smi.nvmlShutdown()

Total memory: 8505131008
Free memory: 6595018752
Used memory: 1910112256
initial boxes: []
No prediction for 0
initial boxes: ['mandatory']
initial boxes: ['mandatory']
initial boxes: ['mandatory', 'warning']
initial boxes: ['mandatory', 'warning']
initial boxes: ['mandatory']
Testing complete!
